# Download vocabulary from NLTK

In [62]:
# %pip install nltk

In [63]:
import nltk
from nltk.corpus import words

nltk.download('words')

[nltk_data] Downloading package words to /home/nico/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Define suggestion function

In [64]:
def load_english_vocabulary(name='en'):
    """
    Download and load the English vocabulary from NLTK.
    
    Return:
        set: A set containing English words.
    """

    vocabulary = set(words.words(name))
    vocabulary = {word.lower() for word in vocabulary}
    print(f"Vocabulary loaded with {len(vocabulary)} words.")
    return vocabulary

In [65]:
def levenshtein_distance(token1, token2):
    """
    Calculate the Levenshtein distance between two tokens.
    
    Return:
        int: The Levenshtein distance between the two tokens.
    """
    
    distances = [[0] * (len(token2) + 1) for _ in range(len(token1) + 1)]

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2

    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if token1[t1 - 1] == token2[t2 - 1]:
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]      # insert
                b = distances[t1 - 1][t2]      # delete
                c = distances[t1 - 1][t2 - 1]  # replace
                distances[t1][t2] = min(a, b, c) + 1

    return distances[len(token1)][len(token2)]

In [66]:
def suggest_corrections(token, 
                        vocabulary, 
                        max_suggestions=5, 
                        max_distance=2):
    """
    Suggest corrections for a given token based on the English vocabulary.
    
    Return:
        List[str]: A list of suggested corrections for the token.
    """
    # If the token is in the vocabulary, it is a right token \(no correction needed)
    token_lower = token.lower()
    if token_lower in vocabulary:
        return [token]

    # Return candidates with small Levenshtein distance
    candidates = []
    for word in vocabulary:
        # Calculate Levenshtein distance only if the \
            # length difference is within the allowed range 
        # This helps to reduce the number of comparisons
        if abs(len(token_lower) - len(word)) <= max_distance:
            dist = levenshtein_distance(token_lower, word)
            if dist <= max_distance:
                candidates.append((word, dist))

    candidates.sort(key=lambda x: x[1])

    return [word for word, dist in candidates[:max_suggestions]]

# Testing with sample texts

In [ ]:
english_vocabulary = load_english_vocabulary()
test_words = ['Ameraca', 'bsn', 'hsllo']

for test in test_words:
    suggestions = suggest_corrections(test, english_vocabulary)
    print(f"Suggestions for '{test}': {suggestions}")

Vocabulary loaded with 234371 words.
Suggestions for 'Ameraca': ['america', 'amerce', 'araca', 'maraca', 'amurca']
Suggestions for 'bsn': ['bin', 'bosn', 'bon', 'ben', 'bun']
Suggestions for 'hallo': ['callo', 'hall', 'halloo', 'hallow', 'hello']


# Calculate keyboard distance

In [68]:
from collections import deque
import pandas as pd

def build_keyboard_graph():
    """
    Create a graph (adjacency list) for the QWERTY keyboard.
    Each key is a node, and its value is a set of adjacent keys.
    
    Return:
        dict: A dictionary representing the keyboard graph.
    """
    layout = [
        "qwertyuiop",
        "asdfghjkl",
        "zxcvbnm"
    ]
    
    graph = {char: set() for row in layout for char in row}
    rows = len(layout)
    
    for r_idx, row in enumerate(layout):
        cols = len(row)
        for c_idx, char in enumerate(row):
            # Add neighbors in the above row
            if r_idx > 0 and c_idx < len(layout[r_idx - 1]):
                neighbor = layout[r_idx - 1][c_idx]
                graph[char].add(neighbor)
                graph[neighbor].add(char)

            # Add neighbors in the left
            if c_idx > 0:
                neighbor = layout[r_idx][c_idx - 1]
                graph[char].add(neighbor)
                graph[neighbor].add(char)
                
    return graph

def bfs_from_start_node(graph, start_node):
    """
    Run BFS from a start node to find distances to all other nodes.
    
    Input:
        graph (dict): The keyboard graph.
        start_node (str): The key from which to start the BFS.
        
    Return:
        dict: A dictionary with distances from the start node to all other nodes.
    """
    # Initialize distances with -1 (unvisited)
    distances = {node: -1 for node in graph}

    # Set the distance to the start node as 0
    distances[start_node] = 0
    
    queue = deque([start_node])
    
    while queue:
        current_node = queue.popleft()
        
        for neighbor in sorted(list(graph[current_node])):
            if distances[neighbor] == -1: # If node is unvisited
                distances[neighbor] = distances[current_node] + 1
                queue.append(neighbor)
                
    return distances

def generate_distance_matrix():
    """
    Generate a distance matrix for all keys on the keyboard.
    """
    keyboard_graph = build_keyboard_graph()
    all_chars = sorted(keyboard_graph.keys())
    
    distance_matrix = {}
    
    for char in all_chars:
        # Chạy BFS từ mỗi phím để lấy một hàng của ma trận
        distance_matrix[char] = bfs_from_start_node(keyboard_graph, char)
        
    return distance_matrix

final_matrix = generate_distance_matrix()

# Save the distance matrix to a csv file
df = pd.DataFrame(final_matrix)

# Sort the DataFrame by index (characters)
df = df.reindex(sorted(df.index))

df.to_csv("keyboard_distances.csv", index_label='char')
print("Distance matrix saved to 'keyboard_distances.csv'")

Distance matrix saved to 'keyboard_distances.csv'


# Add keyboard aware suggestion function

In [69]:
import csv

def load_distances_from_csv(filename="keyboard_distances.csv"):
    """
    Load the keyboard distance matrix from a CSV file.

    Returns:
        dict: A dictionary containing the distances between characters.
    """
    distances = {}

    with open(filename, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        header = next(reader)[1:] # Skip the first column (character names)
        
        for row in reader:
            char1 = row[0]
            char1_distances = {}
            for i, dist_str in enumerate(row[1:]):
                char2 = header[i]
                char1_distances[char2] = int(dist_str)
            distances[char1] = char1_distances
            
    print(f"Keyboard distances loaded from {filename}.")
    return distances

In [70]:
def keyboard_levenshtein_distance(token1, token2, keyboard_distances):
    COST_INSERT = 1.0
    COST_DELETE = 1.0
    
    token1 = token1.lower()
    token2 = token2.lower()
    
    distances = [[0.0] * (len(token2) + 1) for _ in range(len(token1) + 1)]

    for t1 in range(len(token1) + 1):
        distances[t1][0] = float(t1) * COST_DELETE

    for t2 in range(len(token2) + 1):
        distances[0][t2] = float(t2) * COST_INSERT

    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            char1 = token1[t1 - 1]
            char2 = token2[t2 - 1]

            if char1 == char2:
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                key_dist = keyboard_distances.get(char1, {}).get(char2, 99)
                
                if key_dist == 1:
                    cost_replace = 0.8
                elif key_dist == 2:
                    cost_replace = 0.9
                else:
                    cost_replace = 1.0

                cost_del = distances[t1 - 1][t2] + COST_DELETE
                cost_ins = distances[t1][t2 - 1] + COST_INSERT
                cost_sub = distances[t1 - 1][t2 - 1] + cost_replace
                
                distances[t1][t2] = min(cost_del, cost_ins, cost_sub)

    return distances[len(token1)][len(token2)]

In [71]:
def suggest_corrections_improve(token, 
                        vocabulary,
                        keyboard_distances,
                        max_suggestions=5,
                        max_distance=2): 
    """
    Suggest corrections for a given token based on the English vocabulary.
    
    Return:
        List[str]: A list of suggested corrections for the token.
    """
    token_lower = token.lower()
    if token_lower in vocabulary:
        return [token]

    candidates = []
    for word in vocabulary:
        if abs(len(token_lower) - len(word)) <= max_distance:
            dist = keyboard_levenshtein_distance(token_lower, word, keyboard_distances)
            if dist <= max_distance:
                candidates.append((word, dist))
    
    candidates.sort(key=lambda x: x[1])

    return [word for word, dist in candidates[:max_suggestions]]

# Testing with sample texts

In [ ]:
keyboard_distances = load_distances_from_csv("keyboard_distances.csv")

for test in test_words:
    suggestions = suggest_corrections_improve(test, english_vocabulary, keyboard_distances)
    print(f"Đề xuất cho '{test}': {suggestions}")

Keyboard distances loaded from keyboard_distances.csv.
Đề xuất cho 'Ameraca': ['america', 'amerce', 'araca', 'maraca', 'amurca']
Đề xuất cho 'bsn': ['ban', 'ben', 'bin', 'bosn', 'bon']
Đề xuất cho 'hsllo': ['hello', 'hollo', 'sloo', 'callo', 'hall']


# Testing with Birkbeck Spelling Error Corpus

In [73]:
path = "./missp.dat.txt"

with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

pairs = [] # (misspell, correct)
current_correct = None
for token in text.split():
    if token.startswith('$'):
        current_correct = token[1:]
    else:
        pairs.append((token.lower(), current_correct.lower()))

In [79]:
from collections import Counter
from tqdm import tqdm

vocabulary = load_english_vocabulary('en-basic')

def evaluate_improve(pairs, suggester, topk):
    hits = 0
    for mis, cor in tqdm(pairs, desc=f"Evaluating Top-{topk}"):
        suggestions = suggester(mis, vocabulary, keyboard_distances,
                                max_suggestions=topk, max_distance=2)
        top_suggestions = [s.lower() for s in suggestions]
        if cor in top_suggestions[:topk]:
            hits += 1
    return hits / len(pairs)

top1 = evaluate_improve(pairs, suggest_corrections_improve, 1)
top5 = evaluate_improve(pairs, suggest_corrections_improve, 5)

print(f"Top-1 accuracy: {top1:.3%}")
print(f"Top-5 accuracy: {top5:.3%}")

Vocabulary loaded with 850 words.


Evaluating Top-5: 100%|██████████| 36133/36133 [02:10<00:00, 277.66it/s] 

Top-1 accuracy: 4.104%
Top-5 accuracy: 5.975%


In [ ]:
def evaluate(pairs, suggester, topk):
    hits = 0
    for mis, cor in tqdm(pairs, desc=f"Evaluating Top-{topk}"):
        suggestions = suggester(mis, vocabulary,
                                max_suggestions=topk, max_distance=2)
        top_suggestions = [s.lower() for s in suggestions]
        if cor in top_suggestions[:topk]:
            hits += 1
    return hits / len(pairs)

top1 = evaluate(pairs, suggest_corrections, 1)
top5 = evaluate(pairs, suggest_corrections, 5)

print(f"Top-1 accuracy: {top1:.3%}")
print(f"Top-5 accuracy: {top5:.3%}")

Evaluating Top-5: 100%|██████████| 36133/36133 [01:23<00:00, 434.17it/s] 

Top-1 accuracy: 4.173%
Top-5 accuracy: 5.978%
